In [33]:
from google.colab import files
uploaded = files.upload()

Saving SAMPLE.csv to SAMPLE.csv
Saving TEST_FINAL.csv to TEST_FINAL (1).csv
Saving TRAIN.csv to TRAIN (1).csv


In [3]:
import pandas as pd

train_df = pd.read_csv("TRAIN.csv", low_memory=False)
test_df = pd.read_csv("TEST_FINAL.csv")


In [4]:
def split_date(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['WeekOfYear_x'] = df.Date.dt.isocalendar().week

In [5]:
split_date(train_df)
split_date(test_df)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             188340 non-null  object        
 1   Store_id       188340 non-null  int64         
 2   Store_Type     188340 non-null  object        
 3   Location_Type  188340 non-null  object        
 4   Region_Code    188340 non-null  object        
 5   Date           188340 non-null  datetime64[ns]
 6   Holiday        188340 non-null  int64         
 7   Discount       188340 non-null  object        
 8   #Order         188340 non-null  int64         
 9   Sales          188340 non-null  float64       
 10  Year           188340 non-null  int64         
 11  Month          188340 non-null  int64         
 12  Day            188340 non-null  int64         
 13  WeekOfYear_x   188340 non-null  UInt32        
dtypes: UInt32(1), datetime64[ns](1), float64(1), int64(6

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22265 entries, 0 to 22264
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             22265 non-null  object        
 1   Store_id       22265 non-null  int64         
 2   Store_Type     22265 non-null  object        
 3   Location_Type  22265 non-null  object        
 4   Region_Code    22265 non-null  object        
 5   Date           22265 non-null  datetime64[ns]
 6   Holiday        22265 non-null  int64         
 7   Discount       22265 non-null  object        
 8   Year           22265 non-null  int64         
 9   Month          22265 non-null  int64         
 10  Day            22265 non-null  int64         
 11  WeekOfYear_x   22265 non-null  UInt32        
dtypes: UInt32(1), datetime64[ns](1), int64(5), object(5)
memory usage: 2.0+ MB


In [8]:
input_cols=['Store_id', 'Store_Type', 'Location_Type', 'Region_Code', 'Holiday', 
              'Discount', 
              'Day', 'Month', 'Year', 'WeekOfYear_x']
target_col=['Sales']

In [9]:
input_x=train_df[input_cols].copy()
target_x=train_df[target_col].copy()

In [10]:
input_test=test_df[input_cols].copy()

In [21]:
numeric_cols = ['Store_id',
              'Day', 'Month', 'Year', 'WeekOfYear_x',  ]
categorical_cols = ['Store_Type', 'Region_Code', 'Location_Type', 'Discount']

In [22]:
input_x[numeric_cols].isna().sum()

Store_id        0
Day             0
Month           0
Year            0
WeekOfYear_x    0
dtype: int64

In [23]:
input_test[numeric_cols].isna().sum()

Store_id        0
Day             0
Month           0
Year            0
WeekOfYear_x    0
dtype: int64

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(input_x[numeric_cols])

In [25]:
input_x[numeric_cols] = scaler.transform(input_x[numeric_cols])
input_test[numeric_cols] = scaler.transform(input_test[numeric_cols])

In [26]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(input_x[categorical_cols])
encoded_cols = list(encoder.get_feature_names(categorical_cols))

input_x[encoded_cols] = encoder.transform(input_x[categorical_cols])
input_test[encoded_cols] = encoder.transform(input_test[categorical_cols])

In [27]:
X = input_x[numeric_cols + encoded_cols]
X_test = input_test[numeric_cols + encoded_cols]

In [28]:
X_test

,Store_id,Day,Month,Year,WeekOfYear_x,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Discount_No,Discount_Yes
0,0.467033,0.0,0.454545,1.0,0.411765,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.469780,0.0,0.454545,1.0,0.411765,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.472527,0.0,0.454545,1.0,0.411765,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.475275,0.0,0.454545,1.0,0.411765,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.464286,0.0,0.454545,1.0,0.411765,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,0.508242,1.0,0.545455,1.0,0.588235,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
22261,0.027473,1.0,0.545455,1.0,0.588235,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
22262,0.505495,1.0,0.545455,1.0,0.588235,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
22263,0.186813,1.0,0.545455,1.0,0.588235,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [40]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state=42, n_jobs=-1, n_estimators=2500, max_depth=6)

In [41]:
%%time
model.fit(X, target_x)

[11:54:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 15min 40s, sys: 1.35 s, total: 15min 41s
Wall time: 7min 57s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=2500,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [42]:
pred=model.predict(X)
pred

array([12080.802, 44582.887, 34807.977, ..., 30482.639, 48772.977,
       34054.797], dtype=float32)

In [43]:
test_preds = model.predict(X_test)
test_preds

array([75660.91 , 45454.418, 98092.91 , ..., 40190.543, 29180.555,
       31512.324], dtype=float32)

In [44]:
submission_df = pd.read_csv("SAMPLE.csv")

In [45]:
submission_df['Sales']  = test_preds

In [46]:
submission_df

,ID,Sales
0,T1188341,75660.906250
1,T1188342,45454.417969
2,T1188343,98092.906250
3,T1188344,44665.273438
4,T1188345,47693.074219
...,...,...
22260,T1210601,19664.179688
22261,T1210602,54171.628906
22262,T1210603,40190.542969
22263,T1210604,29180.554688


In [47]:
from google.colab import files

submission_df.to_csv('submission_2.csv', index=False)
files.download('submission_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>